## Basic prompty sample
- Chat with prompty
- Add weather function
- http request using natural language
- code interpreter (powershell)

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.22.0"

#r "nuget:AutoGen,0.2.1"
#r "nuget:AutoGen.DotnetInteractive,0.2.1"

Installed Packages AutoGen, 0.2.1 AutoGen.DotnetInteractive, 0.2.1 Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Abstractions, 1.22.0 Microsoft.SemanticKernel.Prompty, 1.22.0-alpha

In [2]:
#r "Interactive.Prompty/Interactive.Prompty/bin/Debug/net8.0/Interactive.Prompty.dll"

In [3]:
using System.Diagnostics;
Debugger.Launch();

In [3]:
using Microsoft.DotNet.Interactive;
using Interactive.Prompty;
using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using AutoGen.DotnetInteractive;
using AutoGen.DotnetInteractive.Extension;


await PromptyExtension.LoadExtensionAsync((CompositeKernel)Microsoft.DotNet.Interactive.Kernel.Root);

In [4]:
#!connect prompty --kernel-name prompty

Kernel added: #!prompty

In [6]:
#!connect prompty-orchestrator --kernel-name orchestrator-kernel --prompty-kernel-name prompty --azure-openai-endpoint @input:{"saveAs":"azure-openai-endpoint"} --azure-openai-deployment-name @input:{"saveAs":"azure-openai-deployment-name"} --azure-openai-apikey @password:{"saveAs":"azure-openai-apikey"}

Please enter a value for parameter: --azure-openai-endpoint Please enter a value for parameter: --azure-openai-deployment-name Please enter a value for parameter: --azure-openai-apikey Ok

Kernel added: #!orchestrator-kernel

In [7]:
---
name: ExamplePrompt
description: A prompt that uses context to ground an incoming question
authors:
  - Seth Juarez
model:
  api: chat
sample:
  firstName: Geno
  context: >
    The Alpine Explorer Tent boasts a detachable divider for privacy,
    numerous mesh windows and adjustable vents for ventilation, and
    a waterproof design. It even has a built-in gear loft for storing
    your outdoor essentials. In short, it's a blend of privacy, comfort,
    and convenience, making it your second home in the heart of nature!
  question: What can you tell me about your tents?
---
 
system:
You are an AI assistant who helps people with_anything. As the assistant,
you answer questions briefly, succinctly, and in a personable manner using
markdown and even add some personal flair with appropriate emojis. You try to resolve customer's question in one-step
 
# Customer
You are helping {{firstName}} to find answers to their questions.
Use their name to address them in your responses.
 
# Context
Use the following context to provide a more personalized response to {{firstName}}:
{{context}}
 
user:
{{input}}

In [8]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.ComponentModel;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.Commands;

[KernelFunction]
[Description("Get the weather for a city and state")]
public static Task<string> GetWeather(
    [Description("The city to get the weather for")] string city,
    [Description("The state to get the weather for")] string state)
{
    return Task.FromResult($"The weather in {city}, {state} is 100 degrees");
}

[KernelFunction]
[Description("shut down your computer")]
public async static Task<string> ShutDownPC()
{
    // System.Diagnostics.Process.Start("shutdown","/s /t 0");

    return "PC has been shut down";
}

[KernelFunction]
[Description("Run Powershell")]
public async static Task<string> Powershell([Description("the powershell script to run")] string pwsh)
{
    var events = new List<KernelEvent>();
    var subscriptions = Microsoft.DotNet.Interactive.Kernel.Current.RootKernel.KernelEvents.Subscribe(e => events.Add(e));
    var result = await Microsoft.DotNet.Interactive.Kernel.Root.RunSubmitCodeCommandAsync(pwsh, "pwsh");
    subscriptions.Dispose();
    var displayValues = events
                        .OfType<DisplayEvent>()
                        .SelectMany(x => x.FormattedValues);

    return string.Join("\n", displayValues.Select(x => x.Value)) ?? "Nothing to display";
}

In [9]:
Hi, what's the weather in newyork

Hey Geno! ☀️ It's a sizzling 100°F in New York right now! Make sure to stay hydrated and cool if you're out and about! 🌊🥤

In [10]:
Who are you helping with

I'm helping you, Geno! 😊 What can I assist you with today?

In [11]:
load all images under C:\images

It looks like you're trying to load images from your local drive. Unfortunately, I can't access files or directories on your computer. However, if you're looking for ways to view or manage those images, I can provide tips or suggest software to use! Just let me know. 😊

In [12]:
Hey who am I

Hey Geno! You're my fantastic user seeking answers today! 😊 What can I help you with?

In [13]:
show me the how many files are in current folder using powershell, list their names

Here are the files in your current folder, Geno:

- Interactive.Prompty
- .gitignore
- basic.prompty
- NuGet.config
- README.md
- sample.ipynb
- ss.py
- z3.dib
- z3.ipynb
- z3.prompty

If you need anything else, just let me know! 😊


Name
----
Interactive.Prompty
.gitignore
basic.prompty
NuGet.config
README.md
sample.ipynb
ss.py
z3.dib
z3.ipynb
z3.prompty



In [2]:
GET https://localhost:7220/weatherforecast

###

GET https://localhost:7220/weatherforecast?date=2023-05-11&location=98006

###

GET https://localhost:7220/weatherforecast HTTP/3

Error: No connection could be made because the target machine actively refused it. (localhost:7220)